In [85]:
# Dependencies
import gmaps
import pandas as pd
import requests
import json
from pprint import pprint

# Google developer API key
from config import gkey

# Access maps with unique API key
gmaps.configure(api_key= gkey)

In [2]:
# Import weather data from csv into data frame and dislay data frame
data= pd.read_csv('../Data_Output/city_weather.csv')
data_df=  pd.DataFrame(data)
data_df.head()

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
0,Fortuna,US,40.5982,-124.1573,50.29,62.0,1.0,12.66
1,Port Hawkesbury,CA,45.6169,-61.3485,34.61,86.0,90.0,26.46
2,Svobodnyy,RU,51.4000,128.1333,35.78,68.0,24.0,15.79
3,Hobart,AU,-42.8794,147.3294,64.69,48.0,0.0,10.36
4,Puerto Ayora,EC,-0.7393,-90.3518,79.00,92.0,75.0,3.00


In [3]:
# Check the data types of the values in the columns
data_df.dtypes

City                 object
Country              object
Latitude            float64
Longitude           float64
Temperature (F)     float64
Humidity (%)        float64
Cloudiness (%)      float64
Wind Speed (mph)    float64
dtype: object

In [45]:
# Customize the size of the figure
figure_layout = {
    'width': '700px',
    'height': '375px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

In [58]:
# Get heat map to display humidity for every city in WeatherPy

locations = data_df[["Latitude", "Longitude"]]
weights = data_df["Humidity (%)"]
fig = gmaps.figure(layout= figure_layout, center=(0,0), zoom_level=2)
heat_layer= gmaps.heatmap_layer(locations, weights= weights, dissipating= False, max_intensity= data_df["Humidity (%)"].max(), point_radius= 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='375px', margin='0 auto 0 auto', padding='1px', wi…

In [56]:
# Narrow data for ideal weather conditons (drop any rows that do not have all conditions):
    # max temperature between 70 and 80
    # wind speed < 10mph
    # zero cloudiness

ideal_weather_df= data_df[(data_df["Temperature (F)"]>=70)  & (data_df["Temperature (F)"]<=80)
                      & (data_df["Wind Speed (mph)"]<=10) & (data_df["Cloudiness (%)"]== 0)
                      ]
ideal_weather_df

,City,Country,Latitude,Longitude,Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph)
71,Terenos,BR,-20.4422,-54.8603,75.20,73.0,0.0,5.75
154,Bauchi,NG,10.5000,10.0000,70.75,18.0,0.0,6.55
229,Dubbo,AU,-32.2500,148.6167,73.40,56.0,0.0,9.22
246,Jatai,BR,-17.8814,-51.7144,71.28,61.0,0.0,3.71
260,Saint-Philippe,RE,-21.3585,55.7679,70.23,73.0,0.0,5.75
314,Joaíma,BR,-16.6542,-41.0306,74.03,59.0,0.0,2.04
315,Caravelas,BR,-17.7125,-39.2481,77.56,75.0,0.0,7.11
321,Miandrivazo,MG,-19.5167,45.4667,73.17,90.0,0.0,2.17
334,Porecatu,BR,-22.7558,-51.3792,74.05,65.0,0.0,9.22
416,Tamworth,AU,-31.1000,150.9333,71.60,56.0,0.0,9.22


In [77]:
# Make list of dictionaries with the followign informationfor each idea city using .itertuples()
    # City Name, Country, Location (Latitude, Longitude)
ideal_cities_list = [
    {"city": city.City, "country": city.Country, "location": f"{city.Latitude},{city.Longitude}"}
    for city in ideal_weather_df.itertuples()
]
ideal_cities_list

[{'city': 'Terenos', 'country': 'BR', 'location': '-20.4422,-54.8603'},
 {'city': 'Bauchi', 'country': 'NG', 'location': '10.5,10.0'},
 {'city': 'Dubbo', 'country': 'AU', 'location': '-32.25,148.6167'},
 {'city': 'Jatai', 'country': 'BR', 'location': '-17.8814,-51.7144'},
 {'city': 'Saint-Philippe', 'country': 'RE', 'location': '-21.3585,55.7679'},
 {'city': 'Joaíma', 'country': 'BR', 'location': '-16.6542,-41.0306'},
 {'city': 'Caravelas', 'country': 'BR', 'location': '-17.7125,-39.2481'},
 {'city': 'Miandrivazo', 'country': 'MG', 'location': '-19.5167,45.4667'},
 {'city': 'Porecatu', 'country': 'BR', 'location': '-22.7558,-51.3792'},
 {'city': 'Tamworth', 'country': 'AU', 'location': '-31.1,150.9333'},
 {'city': 'Paoua', 'country': 'CF', 'location': '7.2427,16.4406'},
 {'city': 'Saint-Joseph', 'country': 'RE', 'location': '-21.3667,55.6167'},
 {'city': 'Port Lincoln', 'country': 'AU', 'location': '-34.7333,135.8667'},
 {'city': 'Vila Velha', 'country': 'BR', 'location': '-20.3297,-40

In [83]:
# Build queries to send API request
results= []
for entry in ideal_cities_list:
    # get information for the request
    base_url= "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params= {
        "location": entry["location"],
        "rankby": "distance",
        "type": "lodging",
        "key": gkey,
    }
    
    # assemble url
    print(f'Retrieving Resulst for {entry["city"]} in {entry["country"]}.')
    response= requests.get(base_url, params= params).json()
    results.append(response)
    


    
    

Retrieving Resulst for Terenos in BR.
Retrieving Resulst for Bauchi in NG.
Retrieving Resulst for Dubbo in AU.
Retrieving Resulst for Jatai in BR.
Retrieving Resulst for Saint-Philippe in RE.
Retrieving Resulst for Joaíma in BR.
Retrieving Resulst for Caravelas in BR.
Retrieving Resulst for Miandrivazo in MG.
Retrieving Resulst for Porecatu in BR.
Retrieving Resulst for Tamworth in AU.
Retrieving Resulst for Paoua in CF.
Retrieving Resulst for Saint-Joseph in RE.
Retrieving Resulst for Port Lincoln in AU.
Retrieving Resulst for Vila Velha in BR.
Retrieving Resulst for Itaunja in IN.
Retrieving Resulst for Abu Dhabi in AE.
Retrieving Resulst for Guaratinga in BR.
Retrieving Resulst for Itapirapuã in BR.
Retrieving Resulst for Mineiros in BR.
Retrieving Resulst for Saint-Louis in RE.
Retrieving Resulst for Ruwi in OM.
Retrieving Resulst for Inverell in AU.
Retrieving Resulst for Saint-Pierre in RE.
Retrieving Resulst for Coxim in BR.


[{'html_attributions': [],
  'results': [{'business_status': 'OPERATIONAL',
    'geometry': {'location': {'lat': -20.4383001, 'lng': -54.8652419},
     'viewport': {'northeast': {'lat': -20.4369231697085,
       'lng': -54.8637366197085},
      'southwest': {'lat': -20.4396211302915, 'lng': -54.8664345802915}}},
    'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
    'name': 'QUINTAS',
    'photos': [{'height': 3024,
      'html_attributions': ['<a href="https://maps.google.com/maps/contrib/111126309806655980886">QUINTAS</a>'],
      'photo_reference': 'ATtYBwKoJPubVj7G-Klsbo_bXvd74yS_pvMUzI7Aeh8xiuPhwLSYNe7D696fSAtdmN_kNzDrqmj3XmodBRpBKhLfBAhGJFsfMgJuA1j2K4eKBXdz02TfmWewP5tezxX9k7pECqgvFa53oi8jIAeR1_BZM8sIoI6ex70Urn2Y1_PZSkkHh6Gp',
      'width': 4032}],
    'place_id': 'ChIJ5ZFkpm8hh5QRi3Am9lNnfeQ',
    'plus_code': {'compound_code': 'H46M+MW Terenos - State of Mato Grosso do Sul, Brazil',
     'global_code': '58F7H46M+MW'},
    'reference': 'ChI

In [86]:
pprint(results)

[{'html_attributions': [],
  'results': [{'business_status': 'OPERATIONAL',
               'geometry': {'location': {'lat': -20.4383001,
                                         'lng': -54.8652419},
                            'viewport': {'northeast': {'lat': -20.4369231697085,
                                                       'lng': -54.8637366197085},
                                         'southwest': {'lat': -20.4396211302915,
                                                       'lng': -54.8664345802915}}},
               'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
               'name': 'QUINTAS',
               'photos': [{'height': 3024,
                           'html_attributions': ['<a '
                                                 'href="https://maps.google.com/maps/contrib/111126309806655980886">QUINTAS</a>'],
                           'photo_reference': 'ATtYBwKoJPubVj7G-Klsbo_bXvd74yS_pvMUzI7Aeh8xiuPhwLSYNe7D696fSA

               'types': ['lodging', 'point_of_interest', 'establishment'],
               'vicinity': 'Rue de la Cour des Freres'},
              {'business_status': 'OPERATIONAL',
               'geometry': {'location': {'lat': -21.35740699999999,
                                         'lng': 55.7528137},
                            'viewport': {'northeast': {'lat': -21.35611936970849,
                                                       'lng': 55.75413818029151},
                                         'southwest': {'lat': -21.3588173302915,
                                                       'lng': 55.75144021970851}}},
               'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
               'name': 'Ferme Auberge Le Ti Planteur',
               'photos': [{'height': 3000,
                           'html_attributions': ['<a '
                                                 'href="https://maps.google.com/maps/contrib/11674136016565

               'photos': [{'height': 3024,
                           'html_attributions': ['<a '
                                                 'href="https://maps.google.com/maps/contrib/115833677662658251973">Phil '
                                                 'Henry</a>'],
                           'photo_reference': 'ATtYBwKClrlo3QyffDOqn6ukganjnbHvlSYkF-8UbTzxAmakrQgfpAqpsdE8LbG1_wVISY1iOB60SGeplOqjsMFTfZjuyTQjrl3fehrmPKMPjz_CRpC_82W1Cet5gJsnHfHw-jFiY3HlXjeR9PEiCG-B37ineHODds7n_RW5GrE9XinsQloN',
                           'width': 4032}],
               'place_id': 'ChIJebCJC2eWCmsR99mFXr9zfLs',
               'plus_code': {'compound_code': 'WW2F+8J West Tamworth NSW, '
                                              'Australia',
                             'global_code': '4RWGWW2F+8J'},
               'rating': 4.6,
               'reference': 'ChIJebCJC2eWCmsR99mFXr9zfLs',
               'scope': 'GOOGLE',
               'types': ['lodging', 'point_of_interest', 'establis

                           'html_attributions': ['<a '
                                                 'href="https://maps.google.com/maps/contrib/100339277129387635838">Hostel '
                                                 'da Penha</a>'],
                           'photo_reference': 'ATtYBwJ4Y8zs-HbVjQI_Q-UGHK68Eu7kw9oycPlYtvlPzlSZUMkkitB0J_QmwyUFqlXc-9tQS1FYBN9Qieib_IiLgUpoYktAxtQJ2LohC8UVjIUheVm7gV3W9nDnoftqGyMZzAmDmCoNXAM72eOmIbt7j96E_YDaw-rPnNeIKfmhGfAo0L0P',
                           'width': 719}],
               'place_id': 'ChIJMW2rGDcWuAARuOzr1h7pBHs',
               'plus_code': {'compound_code': 'MP95+Q3 Centro de Vila Velha, '
                                              'Vila Velha - State of Espírito '
                                              'Santo, Brazil',
                             'global_code': '58FXMP95+Q3'},
               'rating': 3.8,
               'reference': 'ChIJMW2rGDcWuAARuOzr1h7pBHs',
               'scope': 'GOOGLE',
               'ty

               'vicinity': 'Avenida Manoel Ribeiro Carneiro, 39 - Centro, '
                           'Itabela'},
              {'business_status': 'OPERATIONAL',
               'geometry': {'location': {'lat': -16.57213699999999,
                                         'lng': -39.5585891},
                            'viewport': {'northeast': {'lat': -16.5708079697085,
                                                       'lng': -39.5572911197085},
                                         'southwest': {'lat': -16.5735059302915,
                                                       'lng': -39.5599890802915}}},
               'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
               'name': 'Hotel Caraíva',
               'photos': [{'height': 2592,
                           'html_attributions': ['<a '
                                                 'href="https://maps.google.com/maps/contrib/101031706025328127809">Bétto '
               

                           'width': 4032}],
               'place_id': 'ChIJu7wFvNv5kT4Ratmy1BVoIek',
               'plus_code': {'compound_code': 'HGVW+3P Muscat, Oman',
                             'global_code': '7HMWHGVW+3P'},
               'rating': 3.7,
               'reference': 'ChIJu7wFvNv5kT4Ratmy1BVoIek',
               'scope': 'GOOGLE',
               'types': ['lodging', 'point_of_interest', 'establishment'],
               'user_ratings_total': 31,
               'vicinity': 'Muscat'},
              {'business_status': 'OPERATIONAL',
               'geometry': {'location': {'lat': 23.5925049, 'lng': 58.5468062},
                            'viewport': {'northeast': {'lat': 23.5938729302915,
                                                       'lng': 58.5480965802915},
                                         'southwest': {'lat': 23.5911749697085,
                                                       'lng': 58.54539861970849}}},
               'icon': 'https://maps.

               'photos': [{'height': 3000,
                           'html_attributions': ['<a '
                                                 'href="https://maps.google.com/maps/contrib/101766648719398182278">Samuel '
                                                 'A. Carneiro</a>'],
                           'photo_reference': 'ATtYBwIZ2EbMM1zFj-CMRbHQzrVTPbsyCwa_33KAzUdhbuVUFA7D5o3mjCRS6wErii6FJdd9Ss6ov4e18HtseaEk7dTlGvLdY-2Gsgghvaoq2vHeissexAnwFDKHFP4pJxWh52NadXfT0AUHnDnSiAU2mf0jOhV1KbNGv_L0o9qG0UXjbvdA',
                           'width': 4000}],
               'place_id': 'ChIJx9o15Jo1fpMR37UudKHVFaU',
               'plus_code': {'compound_code': 'G63V+QP Vila Santa Marta, Coxim '
                                              '- State of Mato Grosso do Sul, '
                                              'Brazil',
                             'global_code': '58H7G63V+QP'},
               'rating': 4.6,
               'reference': 'ChIJx9o15Jo1fpMR37UudKHVFaU',
          